In [ ]:
# !gdalinfo --version
# !conda install gdal

In [51]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.io import MemoryFile
from rasterio.transform import from_origin
import os
import glob
import shutil
from scipy.ndimage import rotate

In [52]:
from affine import Affine
# Open the TIFF file
def generate_datasest(count_255_th,chip_size,image_3band_path,label_image_path,vv_image_path,vh_image_path,chip_output_path,label_output_path,identifier):
    with rasterio.open(label_image_path) as label_dataset:
        # label_profile = label_dataset.profile
        nodata_value = label_dataset.nodata
        # Read the first band
        label_band = label_dataset.read(1).astype(float) # Read the first band
        label_band[label_band == nodata_value] = np.nan
        print("unique values in label band before: ",np.unique(label_band))
        label_band[label_band < 20] = 0
        print("unique values in label band after: ",np.unique(label_band))

    with rasterio.open(vv_image_path) as vv_dataset:
        profile = vv_dataset.profile
        nodata_value = vv_dataset.nodata
        # Read the first band
        vv_band = vv_dataset.read(1).astype(float) # Read the first band
        # vv_band = rotate(vv_band, angle=-15, reshape=False, mode='nearest')
        vv_band[vv_band == nodata_value] = np.nan

    with rasterio.open(vh_image_path) as vh_dataset:
        nodata_value = vh_dataset.nodata
        # Read the first band
        vh_band = vh_dataset.read(1).astype(float)  # Read the first band
        # vh_band = rotate(vh_band, angle=-15, reshape=False, mode='nearest')
        vh_band[vh_band == nodata_value] = np.nan

    vv_vh_band_ratio = np.where(vh_band != 0, vv_band / vh_band, np.nan)


    profile.update(
        count=3,  # 3 Bands
        dtype="float32",
        nodata=np.nan  # Set nodata to NaN
    )

    with rasterio.open(image_3band_path, "w", **profile) as dst:
        dst.write(vv_band, 1)  # Band 1: VV
        dst.write(vh_band, 2)  # Band 2: VH
        dst.write(vv_vh_band_ratio, 3)

    memfile = MemoryFile()
    with memfile.open(**profile) as dst:
        dst.write(vv_band, 1)  # Band 1: VV
        dst.write(vh_band, 2)  # Band 2: VH
        dst.write(vv_vh_band_ratio, 3)


    # Open in-memory dataset
    with memfile.open() as dataset:
        width, height = dataset.width, dataset.height  # Get image size
        num_bands = dataset.count  # Get the number of bands
        transform = dataset.transform  # Original affine transform
        data = dataset.read()  # Read all bands into a NumPy array (shape: [bands, height, width])

    key=0
    for i in range(0, height, chip_size):  
        for j in range(0, width, chip_size):
            # Ensure we don't go out of bounds
            if i + chip_size <= height and j + chip_size <= width:
                chip = data[:, i:i+chip_size, j:j+chip_size]  # Extract chip
                label=label_band[i:i+chip_size, j:j+chip_size]
                label_unique = np.unique(label)

                if np.array_equal(label_unique, [0]) or \
                (len(label_unique) == 1 and np.isnan(label_unique[0])) or \
                (len(label_unique) == 2 and np.isnan(label_unique[0]) and label_unique[1] == 0):
                    continue
                count_255 = np.count_nonzero(label == 255)

                # Check if count of 255 is less than 50000
                if count_255 < count_255_th:
                    continue

                # Update profile for the chip
                chip_profile = profile.copy()

                # Compute the updated transform for the chip
                chip_transform = transform * Affine.translation(j, i)

                # chip_profile.update(
                #     width=chip_size,
                #     height=chip_size,
                #     transform=chip_transform
                # )

                # # Calculate the new transform for the chip
                # chip_transform = transform * from_origin(
                #     transform.c + j * transform.a,  # Updated top-left x (x_min)
                #     transform.f + i * transform.e,  # Updated top-left y (y_max)
                #     transform.a,  # Pixel width (same for the chip)
                #     transform.e,  # Pixel height (same for the chip)
                # )
                
                chip_profile.update(
                    width=chip_size,
                    height=chip_size,
                    transform=chip_transform  # Set the updated transform
                )
                label_profile=chip_profile.copy()
                label_profile.update(count=1)

                chip_path = f"{chip_output_path}/{identifier}_{key}.tif"
                label_path = f"{label_output_path}/{identifier}_{key}.tif"

                key=key+1
                with rasterio.open(chip_path, "w", **chip_profile) as dst:
                    dst.write(chip)

                with rasterio.open(label_path, "w", **label_profile) as dst:
                    dst.write(label,1)


In [53]:
base_dir="Data/nasa_sar_manual_download/"
folder_names=os.listdir(base_dir)
folder_names.remove('.DS_Store')
folder_names.sort()

for folder_name in folder_names:
    if folder_name.startswith('bangladesh'):
        count_255_th=5000
    elif folder_name.startswith('florence'):
        count_255_th=2500
    elif folder_name.startswith('northal'):
        count_255_th=2500
    elif folder_name.startswith('redrivernorth'):
        count_255_th=500

    folder_path=base_dir+folder_name
    label_image_path = glob.glob(f"{folder_path}/*_label.tif")[0]
    vv_image_path = glob.glob(f"{folder_path}/*_vv.tif")[0]
    vh_image_path = glob.glob(f"{folder_path}/*_vh.tif")[0]

    chip_size = 400
    chip_output_path =f"Data/nasa_sar_manual_download/{folder_name}/images"
    label_output_path=f"Data/nasa_sar_manual_download/{folder_name}/labels"
    image_3band_path=f"Data/nasa_sar_manual_download/{folder_name}/{folder_name}_rgb.tif"
    identifier=folder_name

    if os.path.exists(chip_output_path):
        shutil.rmtree(chip_output_path)
    os.makedirs(chip_output_path)

    if os.path.exists(label_output_path):
        shutil.rmtree(label_output_path)
    os.makedirs(label_output_path)

    if os.path.exists(image_3band_path):
        os.remove(image_3band_path)

    generate_datasest(count_255_th,chip_size,image_3band_path,label_image_path,vv_image_path,vh_image_path,chip_output_path,label_output_path,identifier)
    # break


unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]


/var/folders/z7/dwc0g9dx10n413p0gzrtz1lh0000gn/T/ipykernel_47207/3548898099.py:29: RuntimeWarning: divide by zero encountered in divide
  vv_vh_band_ratio = np.where(vh_band != 0, vv_band / vh_band, np.nan)
/var/folders/z7/dwc0g9dx10n413p0gzrtz1lh0000gn/T/ipykernel_47207/3548898099.py:29: RuntimeWarning: invalid value encountered in divide
  vv_vh_band_ratio = np.where(vh_band != 0, vv_band / vh_band, np.nan)


unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band after:  [  0. 255.]
unique values in label band before:  [  0.  15. 255.]
unique values in label band 

In [21]:
#10000
# vv_image_path="Data/nasa_sar_manual_download/bangladesh_20170314t115609/bangladesh_img_news1a_iw_rt30_20170314t115609_g_gpf_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/bangladesh_20170314t115609/bangladesh_img_news1a_iw_rt30_20170314t115609_g_gpf_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/bangladesh_20170314t115609/bangladesh_20170314t115609_label.tif"

# 10000
# vv_image_path="Data/nasa_sar_manual_download/bangladesh_20170606t115613/bangladesh_img_news1a_iw_rt30_20170606t115613_g_gpf_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/bangladesh_20170606t115613/bangladesh_img_news1a_iw_rt30_20170606t115613_g_gpf_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/bangladesh_20170606t115613/bangladesh_20170606t115613_label.tif"

# 10000
# vv_image_path="Data/nasa_sar_manual_download/bangladesh_20170712t115615/bangladesh_img_news1a_iw_rt30_20170712t115615_g_gpf_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/bangladesh_20170712t115615/bangladesh_img_news1a_iw_rt30_20170712t115615_g_gpf_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/bangladesh_20170712t115615/bangladesh_20170712t115615_label.tif"

#5000
# vv_image_path="Data/nasa_sar_manual_download/florence_20180510t231343/florence_img_s1a_iw_rt30_20180510t231343_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/florence_20180510t231343/florence_img_s1a_iw_rt30_20180510t231343_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/florence_20180510t231343/florence_20180510t231343_label.tif"


#5000
# vv_image_path="Data/nasa_sar_manual_download/florence_20180522t231344/florence_img_s1a_iw_rt30_20180522t231344_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/florence_20180522t231344/florence_img_s1a_iw_rt30_20180522t231344_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/florence_20180522t231344/florence_20180522t231344_label.tif"


# vv_image_path="Data/nasa_sar_manual_download/florence_20180919t231350/florence_img_s1a_iw_rt30_20180919t231350_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/florence_20180919t231350/florence_img_s1a_iw_rt30_20180919t231350_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/florence_20180919t231350/florence_20180919t231350_label.tif"

# 15000
# vv_image_path="Data/nasa_sar_manual_download/florence_20180721t231347/florence_img_s1a_iw_rt30_20180721t231347_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/florence_20180721t231347/florence_img_s1a_iw_rt30_20180721t231347_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/florence_20180721t231347/florence_20180721t231347_label.tif"
#validation, 5000
# vv_image_path="Data/nasa_sar_manual_download/florence_20180802t231348/florence_img_s1a_iw_rt30_20180802t231348_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/florence_20180802t231348/florence_img_s1a_iw_rt30_20180802t231348_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/florence_20180802t231348/florence_20180802t231348_label.tif"

#5000
# vv_image_path="Data/nasa_sar_manual_download/northal_20190302t234651/northal_img_s1a_iw_rt30_20190302t234651_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/northal_20190302t234651/northal_img_s1a_iw_rt30_20190302t234651_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/northal_20190302t234651/northal_20190302t234651_label.tif"

# 15000
# vv_image_path="Data/nasa_sar_manual_download/northal_20190419t234652/northal_img_s1a_iw_rt30_20190419t234652_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/northal_20190419t234652/northal_img_s1a_iw_rt30_20190419t234652_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/northal_20190419t234652/northal_20190419t234652_label.tif"

# 10000
# vv_image_path="Data/nasa_sar_manual_download/northal_20191004t234700/northal_img_s1a_iw_rt30_20191004t234700_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/northal_20191004t234700/northal_img_s1a_iw_rt30_20191004t234700_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/northal_20191004t234700/northal_20191004t234700_label.tif"

#5000
# vv_image_path="Data/nasa_sar_manual_download/northal_20191121t234700/northal_img_s1a_iw_rt30_20191121t234700_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/northal_20191121t234700/northal_img_s1a_iw_rt30_20191121t234700_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/northal_20191121t234700/northal_20191121t234700_label.tif"

# validation,5000
# vv_image_path="Data/nasa_sar_manual_download/northal_20191227t234659/northal_img_s1a_iw_rt30_20191227t234659_g_gpn_vh.tif"
# vh_image_path="Data/nasa_sar_manual_download/northal_20191227t234659/northal_img_s1a_iw_rt30_20191227t234659_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/northal_20191227t234659/northal_20191227t234659_label.tif"


#1000
# vv_image_path="Data/nasa_sar_manual_download/redrivernorth_20190104t002247/redrivernorth_img_s1b_iw_rt30_20190104t002247_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/redrivernorth_20190104t002247/redrivernorth_img_s1b_iw_rt30_20190104t002247_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/redrivernorth_20190104t002247/redrivernorth_20190104t002247_label.tif"

# 1000
# vv_image_path="Data/nasa_sar_manual_download/redrivernorth_20190410t002246/redrivernorth_img_s1b_iw_rt30_20190410t002246_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/redrivernorth_20190410t002246/redrivernorth_img_s1b_iw_rt30_20190410t002246_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/redrivernorth_20190410t002246/redrivernorth_20190410t002246_label.tif"

# validation, 1000
# vv_image_path="Data/nasa_sar_manual_download/redrivernorth_20190621t002250/redrivernorth_img_s1b_iw_rt30_20190621t002250_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/redrivernorth_20190621t002250/redrivernorth_img_s1b_iw_rt30_20190621t002250_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/redrivernorth_20190621t002250/redrivernorth_20190621t002250_label.tif"




In [25]:

# vv_image_path="Data/nasa_sar_manual_download/florence_20180802t231348/florence_img_s1a_iw_rt30_20180802t231348_g_gpn_vv.tif"
# vh_image_path="Data/nasa_sar_manual_download/florence_20180802t231348/florence_img_s1a_iw_rt30_20180802t231348_g_gpn_vh.tif"
# label_image_path="Data/nasa_sar_manual_download/florence_20180802t231348/florence_20180802t231348_label.tif"

# # Define chip size
# chip_size = 400
# # count_255_th = 5000
# chip_output_path ="Data/nasa_sar_manual_download/florence_20180522t231344/images"
# label_output_path="Data/nasa_sar_manual_download/florence_20180522t231344/labels"
# # identifier="20180522t231344"
# image_3band_path=f"Data/nasa_sar_manual_download/florence_20180522t231344/{identifier}_rgb.tif"



In [ ]:
# import rasterio
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.ndimage import rotate


# # Open TIFF file
# with rasterio.open(vv_image_path) as dataset:
#     band = dataset.read(1).astype(float)  # Convert to float for NoData handling
#     nodata_value = dataset.nodata  # Get NoData value

#     # Rotate image by 45 degrees (or any angle you prefer)
#     rotated_band = rotate(band, angle=-15, reshape=True, mode='nearest')

#     # Replace NoData values (if needed)
#     if nodata_value is not None:
#         rotated_band[rotated_band == nodata_value] = np.nan

#     # Show rotated image
#     plt.figure(figsize=(8, 6))
#     plt.imshow(rotated_band, cmap="gray")
#     plt.colorbar(label="Pixel Values")
#     plt.title("Rotated TIFF Image")
#     plt.show()

#     # Update metadata for saving
#     new_meta = dataset.meta.copy()
#     new_meta.update({"height": rotated_band.shape[0], "width": rotated_band.shape[1]})



# print("Rotated image saved successfully!")


In [ ]:
# with rasterio.open(vv_image_path) as dataset:
#     transform = dataset.transform  # Affine transformation matrix
#     print("Affine Transform:", transform)

#     # Extract rotation parameters
#     a, b, d, e = transform.a, transform.b, transform.d, transform.e

#     # Compute rotation angle in degrees
#     rotation_angle = np.arctan2(-b, a) * 180 / np.pi  # atan2 helps find the correct quadrant

#     print(f"Estimated Rotation Angle: {rotation_angle:.2f} degrees")